<img src="idal-logo.png" align="right" style="float" width="400">
<font color="#CA3532"><h1 align="left">mIA3</h1></font>
<font color="#6E6E6E"><h2 align="left">Tarea Evaluable. Aprendizaje Profundo 1 y 2 (Parte 2 de 3).</h2></font> 

#### Elaborado por Felipe Ramírez Herrera

In [ ]:
# !pip install numpy
# !pip install panda

# !pip install torch
# !pip install torchtext
# !pip install torchmetrics
# !pip install spacy

# !pip install jupyter
# !pip install ipywidgets

# !pip install editdistance
# !pip install six
# !pip install typeguard

# !pip install matplotlib
# !pip install seaborn
# !pip install scikit-learn

# !python -m spacy download en_core_web_sm
# !python -m spacy download es_core_news_sm
# !python -m spacy download fr_core_news_sm
# !pip install wget

# !pip install --upgrade --force-reinstall torchtext

In [ ]:
import math
import random
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing as mp
import tqdm
import numpy as np
import os
import pathlib
import csv
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader, DistributedSampler
from torchtext.vocab import Vocab
import logging
import pandas as pd
import six
from typing import List, Tuple, Union
from argparse import Namespace
from tqdm.notebook import tqdm

import matplotlib.pyplot as mp
from matplotlib.colors import ListedColormap
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split


warnings.filterwarnings('ignore')

In [ ]:
# Formato de Mathplotlib

mp.rc('font', size=8)
mp.rc('axes', titlesize=8)
mp.rc('axes', labelsize=8)
mp.rc('xtick', labelsize=8)
mp.rc('ytick', labelsize=8)
mp.rc('legend', fontsize=8)
mp.rc('figure', titlesize=8)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"


In [ ]:
torch.manual_seed(1234)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    !nvidia-smi

In [ ]:
def get_cuda_memory_available():
    if torch.cuda.is_available():
        t = torch.cuda.get_device_properties(DEVICE).total_memory
        r = torch.cuda.memory_reserved(DEVICE)
        a = torch.cuda.memory_allocated(DEVICE)
        f = (t - a - r) / t  # free inside reserved
        print("CUDA Available Memory: {0}".format(f))

In [ ]:
def count_params(model, return_int=False):
    params = sum([torch.prod(torch.tensor(x.shape)).item() for x in model.parameters() if x.requires_grad])
    if return_int:
        return params
    else:
        print("There are {:,} trainable parameters in this model.".format(params))

In [ ]:
# https://conferences.unite.un.org/UNCorpus/Home/DownloadOverview
# !cat /home/framirez/translation_multilingual/UNv1.0.6way.tar.gz.* | tar -xzf -

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import zipfile

# EN_corpus_file = "UNv1.0.6way_en.zip" # UN Parallel Corpus English (EN)
# ES_corpus_file = "UNv1.0.6way_es.zip" # UN Parallel Corpus Spanish (ES)
# FR_corpus_file = "UNv1.0.6way_fr.zip" # UN Parallel Corpus French (FR)

# corpus_source_folder = "/content/drive/MyDrive/"
# corpus_target_folder = "/tmp/"

# if (not os.path.exists(corpus_target_folder + EN_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + EN_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)

# if (not os.path.exists(corpus_target_folder + ES_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + ES_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)

# if (not os.path.exists(corpus_target_folder + FR_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + FR_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)


In [ ]:
# Use the cuDNN backend for 16-bit training and inference for CUDA-enabled GPUs
# torch.backends.cudnn.benchmark =  True
# torch.backends.cudnn.enabled =  True

In [ ]:
max_seq_length = 64 # Re-calculated further 
min_seq_length = 16 # Re-calculated further 

In [ ]:

NUMBER_OF_EPOCHS = 2
EARLY_STOPPING_EPOCHS = 3
CLIPPING_VALUE = 15                   # clipping value, or None to prevent gradient clipping
MAXIMUM_NUMBER_OF_SAMPLES = 70000
BATCH_SIZE = 32

# Lets GPU to get more cooler
INNER_GPU_REST_TIME = 0 # Rest time between epochs
OUTER_GPU_REST_TIME = 0 # Rest time between model training / validation processes


In [ ]:
import pandas as pd  

def generate_examples(src_file, tgt_a_file, tgt_b_file):
    data = []
    with open(src_file, encoding="utf-8") as src_f, open(tgt_a_file, encoding="utf-8") as tgt_a, open(tgt_b_file, encoding="utf-8") as tgt_b:
        for idx, (a, b, c) in enumerate(zip(src_f, tgt_a, tgt_b)):           
            if (a.isspace() | b.isspace() | c.isspace()):
                continue
            data.append({'text_en' : a, 'text_es' : b, 'text_fr' :c} )
    return pd.DataFrame.from_records(data=data)


In [ ]:
import re
from unicodedata import normalize

PAD_WORD = '<pad>'
UNK_WORD = '<unk>'  # Unknown token symbol
BOS_WORD = '<bos>'  # Begin-of-Sentence token symbol
EOS_WORD = '<eos>'  # End-of-Sentence token symbol

def clean_text(text : str):
    text = normalize('NFD', str(text).lower())
    text = re.sub('[^A-Za-z ]+', '', text)
    return text

pkl_dataset_file = "transformers_us_ds.pkl"
un_ds = pd.DataFrame()

if os.path.exists(pkl_dataset_file):
    un_ds = pd.read_pickle(pkl_dataset_file) 
else:
    en_train_path = "./mnt/drive/UNv1.0.6way.en"
    es_train_path = "./mnt/drive/UNv1.0.6way.es"
    fr_train_path = "./mnt/drive/UNv1.0.6way.fr"
    df = generate_examples(en_train_path, es_train_path, fr_train_path)
    df['text_en'] = df['text_en'].apply(lambda row: clean_text(row))
    df['text_es'] = df['text_es'].apply(lambda row: clean_text(row))
    df['text_fr'] = df['text_fr'].apply(lambda row: clean_text(row))
    un_ds = df
    un_ds.to_pickle(pkl_dataset_file)
len(un_ds)


In [ ]:
un_ds.head(25)

In [ ]:
un_ds.head(-25)

In [ ]:
un_ds = un_ds.head(MAXIMUM_NUMBER_OF_SAMPLES)

In [ ]:
from enum import Enum

import torchtext

en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
es_tokenizer = get_tokenizer('spacy', language='es_core_news_sm')
fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')

Language = Enum('Language', ['EN', 'ES', 'FR'])

def yield_tokens(Lang: Language=Language.EN):
    for index, row in un_ds.iterrows():     
         if (Lang == Language.EN):
            yield en_tokenizer(str(row["text_en"]))  
         if (Lang == Language.ES):
            yield es_tokenizer(str(row["text_es"]))
         if (Lang == Language.FR):
           yield fr_tokenizer(str(row["text_fr"]))       

In [ ]:
en_vocab_file = 'transformers_vocab_en.pth'
es_vocab_file = 'transformers_vocab_es.pth'
fr_vocab_file = 'transformers_vocab_fr.pth'

In [ ]:
import torchtext
import torchtext.data
import torchtext.data.datasets_utils
import torchtext.datasets

SPECIALS = [PAD_WORD, BOS_WORD, EOS_WORD,  UNK_WORD]

en_vocabulary = None
es_vocabulary = None
fr_vocabulary = None

if os.path.exists(en_vocab_file):
    en_vocabulary = torch.load(en_vocab_file)
else:
    en_vocabulary = build_vocab_from_iterator(yield_tokens(Language.EN), specials=SPECIALS)
    torch.save(en_vocabulary, en_vocab_file)

if os.path.exists(es_vocab_file):
    es_vocabulary = torch.load(es_vocab_file)
else:
    es_vocabulary = build_vocab_from_iterator(yield_tokens(Language.ES), specials=SPECIALS)
    torch.save(es_vocabulary, es_vocab_file)

if os.path.exists(fr_vocab_file):
    fr_vocabulary = torch.load(fr_vocab_file)
else:
    fr_vocabulary = build_vocab_from_iterator(yield_tokens(Language.FR), specials=SPECIALS)
    torch.save(fr_vocabulary, fr_vocab_file)

In [ ]:
print("EN Vocabulary Size = {0}".format(len(en_vocabulary)))
print("ES Vocabulary Size = {0}".format(len(es_vocabulary)))
print("FR Vocabulary Size = {0}".format(len(fr_vocabulary)))

In [ ]:
PAD_EN_IDX = en_vocabulary[PAD_WORD]
BOS_EN_IDX = en_vocabulary[BOS_WORD]
EOS_EN_IDX = en_vocabulary[EOS_WORD]
UNK_EN_IDX = en_vocabulary[UNK_WORD]


PAD_ES_IDX = es_vocabulary[PAD_WORD]
BOS_ES_IDX = es_vocabulary[BOS_WORD]
EOS_ES_IDX = es_vocabulary[EOS_WORD]
UNK_ES_IDX = es_vocabulary[UNK_WORD]

PAD_FR_IDX = fr_vocabulary[PAD_WORD]
BOS_FR_IDX = fr_vocabulary[BOS_WORD]
EOS_FR_IDX = fr_vocabulary[EOS_WORD]
UNK_FR_IDX = fr_vocabulary[UNK_WORD]

en_vocab_size = len(en_vocabulary) 
es_vocab_size = len(es_vocabulary) 
fr_vocab_size = len(fr_vocabulary)

print("EN: PAD = {0} BOS = {1} EOS = {2} UNK = {3}".format(PAD_EN_IDX, BOS_EN_IDX, EOS_EN_IDX, UNK_EN_IDX))
print("ES: PAD = {0} BOS = {1} EOS = {2} UNK = {3}".format(PAD_ES_IDX, BOS_ES_IDX, EOS_ES_IDX, UNK_ES_IDX))
print("FR: PAD = {0} BOS = {1} EOS = {2} UNK = {3}".format(PAD_FR_IDX, BOS_FR_IDX, EOS_FR_IDX, UNK_FR_IDX))


In [ ]:
def pad_or_truncate(tokenized_text, allocate : bool = True,  pad_index : int = PAD_EN_IDX, bos_index : int = BOS_EN_IDX, eos_index : int = EOS_EN_IDX):  
    result = []
    if len(tokenized_text) < max_seq_length:
        if (allocate):
            result = [bos_index] + tokenized_text + [eos_index]
            left = max_seq_length - len(result)
            padding = [pad_index] * left
            result = result + padding
        else:
            left = max_seq_length - len(tokenized_text)
            padding = [pad_index] * left
            result = tokenized_text + padding       
    else:
        raise Exception("pad_or_truncate: max_seq_length not computed properly")
    return result

In [ ]:
full_data_file = 'transformers_full.pth'
trn_data_file = 'transformers_trn.pth'
val_data_file = 'transformers_val.pth'
tst_data_file = 'transformers_tst.pth'

In [ ]:
len(un_ds)

In [ ]:
import random
import pickle
from collections import Counter 

MINIMUM_ALLOWED_SIZE_OF_SEQ = 5
MAXIMUM_ALLOWED_SIZE_OF_SEQ = 100

full_data = []
trn_subset, val_subset, tst_subset = [], [], []

random.seed(1234)
torch.manual_seed(1234)

max_len = 0
min_len = 4500

en_counter = Counter()
es_counter = Counter()
fr_counter = Counter()

en_lengths = []
es_lengths = []
fr_lengths = []

fe = os.path.exists(full_data_file)
te = os.path.exists(trn_data_file)
ve = os.path.exists(val_data_file)
tt = os.path.exists(tst_data_file)

if fe and te and ve and tt:
    with open(full_data_file, 'rb') as fp:
            full_data = pickle.load(fp)
    with open(trn_data_file, 'rb') as fp:
            trn_subset = pickle.load(fp)
    with open(val_data_file, 'rb') as fp:
            val_subset = pickle.load(fp)        
    with open(tst_data_file, 'rb') as fp:
            tst_subset = pickle.load(fp)           
    for (en_seq,es_seq,fr_seq) in full_data:     
        en_counter.update(en_seq)
        es_counter.update(es_seq)
        fr_counter.update(fr_seq)
        a = len(en_seq)
        b = len(es_seq)
        c = len(fr_seq) 
        en_lengths.append(a)
        es_lengths.append(b)
        fr_lengths.append(c)
        max_len = max(max_len, a, b, c)    
        min_len = min(min_len, a, b, c)    
else:
    for idx, row in un_ds.iterrows():
        
        en_exp = row["text_en"].rstrip("\n")
        es_exp = row["text_es"].rstrip("\n")
        fr_exp = row["text_fr"].rstrip("\n")

        en_seq = [en_vocabulary[token] for token in en_tokenizer(en_exp)]
        es_seq = [es_vocabulary[token] for token in es_tokenizer(es_exp)]
        fr_seq = [fr_vocabulary[token] for token in fr_tokenizer(fr_exp)]

        a = len(en_seq)
        b = len(es_seq)
        c = len(fr_seq) 

        seq_min = min(a, b, c)
        seq_max = max(a, b, c)

        if seq_min  >= MINIMUM_ALLOWED_SIZE_OF_SEQ and seq_max <= MAXIMUM_ALLOWED_SIZE_OF_SEQ: 
            
            en_counter.update(en_seq)
            es_counter.update(es_seq)
            fr_counter.update(fr_seq)
       
            en_lengths.append(a)
            es_lengths.append(b)
            fr_lengths.append(c)

            max_len = max(max_len, seq_max)
            min_len = min(min_len, seq_min) 
            full_data.append((en_seq, es_seq, fr_seq))

    trn_subset, val_subset = train_test_split(full_data, test_size=0.3, train_size=0.7, random_state=1234, shuffle=True)
    val_subset, tst_subset = train_test_split(val_subset, test_size=0.33, train_size=0.67, random_state=1234, shuffle=True)

    with open(full_data_file, 'wb') as fp:
        pickle.dump(full_data, fp)
    with open(trn_data_file, 'wb') as fp:
        pickle.dump(trn_subset, fp)
    with open(val_data_file, 'wb') as fp:
        pickle.dump(val_subset, fp)
    with open(tst_data_file, 'wb') as fp:
        pickle.dump(tst_subset, fp)

if (max_seq_length < max_len):
    max_seq_length = max_len + 2

if (min_seq_length > min_len):
    min_seq_length = min_len + 2


print("Selected Records: {0}".format(len(full_data)))
print("MAX SEQ LEN {0}".format(max_seq_length))
print("MIN SEQ LEN {0}".format(min_seq_length))

size_of_trn_set_size = len(trn_subset)
size_of_val_set_size = len(val_subset)
size_of_tst_set_size = len(tst_subset)

print("Training tuples: {0} Validation tuples: {1} Testing tuples: {2}".format(size_of_trn_set_size, size_of_val_set_size, size_of_tst_set_size))


In [ ]:
fig = mp.figure(figsize=(8, 10))
ax0 = fig.add_subplot(311)
ax0.hist(en_lengths, rwidth=0.8, color='gray')
ax0.set_title("Longitud de las oraciones (EN)")
ax0.set_xlabel("Cantidad de tokens")

ax1 = fig.add_subplot(312)
ax1.hist(es_lengths, rwidth=0.8, color='gray')
ax1.set_title("Longitud de las oraciones (ES)")
ax1.set_xlabel("Cantidad de tokens")

ax2 = fig.add_subplot(313)
ax2.hist(fr_lengths, rwidth=0.8, color='gray')
ax2.set_title("Longitud de las oraciones (FR)")
ax2.set_xlabel("Cantidad de tokens")

mp.tight_layout()
mp.show()

In [ ]:
mp.figure(figsize=(8,6))
mp.hist2d(en_lengths, es_lengths, bins=max_seq_length-2, cmap='binary')
mp.title("Joint Distribution of Sentence Lengths")
mp.xlabel("# English Tokens")
mp.ylabel("# Spanish Tokens")
mp.colorbar()
mp.show()

In [ ]:
mp.figure(figsize=(8,6))
mp.hist2d(en_lengths, fr_lengths, bins=max_seq_length-2, cmap='binary')
mp.title("Joint Distribution of Sentence Lengths")
mp.xlabel("# English Tokens")
mp.ylabel("# French Tokens")
mp.colorbar()
mp.show()

In [ ]:
def plot_top_words(counter : Counter, vocab: Vocab, k=20, ax=None):
    top_k = counter.most_common(k)
    tokens, freqs = zip(*reversed(top_k))
    
    words = [vocab.lookup_token(token) for token in tokens]


    if ax is None:
        mp.barh(words, freqs, color='gray')
    else:
        ax.barh(words, freqs, color='gray')

In [ ]:

fig = mp.figure(figsize=(14, 8))

ax0 = fig.add_subplot(131)
plot_top_words(en_counter, en_vocabulary, ax=ax0)
ax0.set_title("Top 20 English Words")
ax0.set_xlabel("Raw Frequency")

ax1 = fig.add_subplot(132)
plot_top_words(es_counter,es_vocabulary, ax=ax1)
ax1.set_title("Top 20 Spanish Words")
ax1.set_xlabel("Raw Frequency")

ax2 = fig.add_subplot(133)
plot_top_words(fr_counter,fr_vocabulary, ax=ax2)
ax2.set_title("Top 20 French Words")
ax2.set_xlabel("Raw Frequency")

mp.tight_layout()
mp.show()

In [ ]:
import itertools

from  torch.nn.utils.rnn import pad_sequence

for en_id, es_id, fr_id in zip(en_vocabulary.lookup_indices(SPECIALS), es_vocabulary.lookup_indices(SPECIALS), fr_vocabulary.lookup_indices(SPECIALS)):
  assert en_id == es_id & es_id == fr_id

# Importante: Solo cargo los datos al dispositivo (GPU) cuando voy a procesarlos
# es un trade-off entre performance del entrenamiento y capacidad de carga de los
# datos dada las limitaciones de los GPU de consumo (Nvidia RTX):

def tensor_transform(token_ids: List[int], bos_idx, eos_idx): 
    list = [bos_idx] + token_ids + [eos_idx]                         
    return torch.as_tensor(list, device=DEVICE)

def generate_batch(data_batch):
    en_batch, es_batch, fr_batch = [], [], []
    for (en_item, es_item, fr_item) in data_batch:     
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        es_t = tensor_transform(es_item, BOS_ES_IDX, EOS_ES_IDX)
        fr_t = tensor_transform(fr_item, BOS_FR_IDX, EOS_FR_IDX)     
        en_batch.append(en_t)
        es_batch.append(es_t)
        fr_batch.append(fr_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    es_batch = pad_sequence(es_batch, padding_value=PAD_ES_IDX, batch_first=False)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_FR_IDX, batch_first=False)
    return en_batch, es_batch, fr_batch

def generate_batch_EN_ES(data_batch):
    en_batch, es_batch = [], []
    for (en_item, es_item, _) in data_batch:     
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        es_t = tensor_transform(es_item, BOS_ES_IDX, EOS_ES_IDX)
        en_batch.append(en_t)
        es_batch.append(es_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    es_batch = pad_sequence(es_batch, padding_value=PAD_ES_IDX, batch_first=False)
    return en_batch, es_batch

def generate_batch_EN_FR(data_batch):
    en_batch, fr_batch = [], []
    for (en_item, _, fr_item) in data_batch:        
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        fr_t = tensor_transform(fr_item, BOS_FR_IDX, EOS_FR_IDX)
        en_batch.append(en_t)
        fr_batch.append(fr_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_FR_IDX, batch_first=False)
    return en_batch, fr_batch

In [ ]:


class LanguageDataset(Dataset):
    
    def __init__(self, subset, length):

        super().__init__()

        self.length = length
        self.subset = subset

    def __getitem__(self, idx):
        return self.subset[idx]

    def __len__(self):
        return self.length
    





In [ ]:


Bilingual_EN_to_ES_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch_EN_ES, num_workers=16, pin_memory=True)
Bilingual_EN_to_ES_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_ES, num_workers=8, pin_memory=True)
Bilingual_EN_to_ES_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_ES, num_workers=8, pin_memory=True)

Bilingual_EN_to_FR_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch_EN_FR, num_workers=16, pin_memory=True)
Bilingual_EN_to_FR_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_FR, num_workers=8, pin_memory=True)
Bilingual_EN_to_FR_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_FR, num_workers=8, pin_memory=True)

Trilingual_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch, num_workers=16, pin_memory=True)
Trilingual_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch, num_workers=8, pin_memory=True)
Trilingual_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch, num_workers=8, pin_memory=True)

batches_for_training = math.ceil(size_of_trn_set_size / BATCH_SIZE)
batches_for_validation = math.ceil(size_of_val_set_size / BATCH_SIZE)

In [ ]:
import sys

size_to_MB = 1024 * 1024

def getSizeOf(a : torch.Tensor):
    return sys.getsizeof(a) + torch.numel(a) * a.element_size()

def ElementsOf(a : torch.Tensor):
    return torch.numel(a)





In [ ]:
mp.set_start_method('spawn', force=True)

In [ ]:
from torch.utils.data import Dataset, DataLoader, DistributedSampler

def calculate_dataset_memory_overhead(rank, data):

    print("Data Processs ID: {}".format(rank))

    total = 0

    for (x, tgt_ES, tgt_FR) in iter(data):

        total += getSizeOf(x) / size_to_MB
        total += getSizeOf(tgt_ES) / size_to_MB
        total += getSizeOf(tgt_FR) / size_to_MB
        del x
        del tgt_ES
        del tgt_FR

    torch.cuda.empty_cache()

    print("Tamaño estimado del conjunto de datos: {} Mbytes".format(total))


class ParallelLoader(DataLoader):
    def __init__(self, dataset, parallelization_degree, rank, collate_fn, sampler):
        super().__init__(dataset=dataset, sampler = sampler , collate_fn = collate_fn, batch_size=BATCH_SIZE)



parallelization_degree = 8


if __name__ == '__main__':

    processes = []

    for rank in range(1, parallelization_degree):

        sampler = DistributedSampler(dataset=trn_subset, num_replicas=parallelization_degree, rank= rank)

        data = ParallelLoader(LanguageDataset(trn_subset, size_of_trn_set_size), parallelization_degree, rank, generate_batch_EN_ES, sampler)

        p = mp.Process(target=calculate_dataset_memory_overhead, args=[rank, data])
        p.start()
        processes.append(p)

    for p in processes:
        p.join()




torch.cuda.empty_cache()
# torch.multiprocessing.spawn.spawn(calculate_dataset_memory_overhead, args=(), nprocs=8, join=True, daemon=False, start_method='spawn')

In [ ]:
def init_transformer_model(model : nn.Module):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

In [ ]:
def masked_accuracy(label, pred, tgt_pad_idx = PAD_ES_IDX):
  pred = torch.argmax(pred, dim=-1)
  match = label.eq(pred)
  mask = label.ne(tgt_pad_idx)
  match = match & mask
  match = match.type(torch.float32) 
  mask =  mask.type(torch.float32)
  return torch.sum(match)/torch.sum(mask)

In [ ]:
import time
def let_gpu_rest(minutes):
    if minutes > 0:
        time.sleep(minutes * 60)    

In [ ]:
# Fragmento de código. Funciones que despliegan las curvas de desempeño para el modelo
# Se reutilizan a lo largo del ejercicio.

import matplotlib.pyplot as plt



def plot_accuracy_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_acc'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_acc'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()

def plot_loss_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_loss'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_loss'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()

def plot_pplx_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Perplejidad del modelo {0}'.format(name))
    ax.plot(history['train_ppl'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_ppl'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Perplejidad')
    ax.legend()
    plt.show()


def plot_accuracy_curve_dual_transformer(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_conjugate_acc'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_conjugate_acc'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()


def plot_loss_curve_dual_transformer(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_conjugate_loss'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_conjugate_loss'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()

def plot_accuracy_curve_dual_transformer_both(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_acc_a'], label='Entrenamiento (EN-ES)')
    ax.plot(history['train_acc_b'], label='Entrenamiento (EN-FR)')
    ax.plot(history['valid_acc_a'], label='Validación (EN-ES)')
    ax.plot(history['valid_acc_b'], label='Validación (EN-FR)')
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()

def plot_loss_curve_dual_transformer_both(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_loss_a'], label='Entrenamiento (EN-ES)')
    ax.plot(history['train_loss_b'], label='Entrenamiento (EN-FR)')
    ax.plot(history['valid_loss_a'], label='Validación (EN-ES)')
    ax.plot(history['valid_loss_b'], label='Validación (EN-FR)')
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()


# Seq-to-Seq con GCNN

Véase: https://arxiv.org/abs/1612.08083

In [ ]:
INPUT_ES_VOCAB_DIM = en_vocab_size
OUTPUT_ES_VOCAB_DIM = es_vocab_size
OUTPUT_FR_VOCAB_DIM = fr_vocab_size
GCNN_EMBDIM = 256
GCNN_HID_DIM = 512 # each conv. layer has 2 * hid_dim filters
GCNN_ENC_LAYERS = 5 # number of conv. blocks in encoder
GCNN_DEC_LAYERS = 5 # number of conv. blocks in decoder
GCNN_ENC_KERNEL_SIZE = 3 # must be odd!
GCNN_DEC_KERNEL_SIZE = 3 # can be even or odd
GCNN_ENC_DROPOUT = 0.25
GCNN_DEC_DROPOUT = 0.25

CNN_TRG_PAD_IDX = PAD_ES_IDX

In [ ]:
CNN_EN_ES_model_path = 'gcnn_mt_en_es_{0}.pt'
CNN_EN_FR_model_path = 'gcnn_mt__en_fr_{0}.pt'
CNN_EN_ES_FR_model_path = 'gcnn_mt__en_es_fr_{0}.pt'

### Model building blocks

In [ ]:
# Esta función genera una matriz de tensores que representan las posiciones en una secuencia. 
# Es una función auxiliar utilizada para crear las embeddings posicionales necesarias para el modelo
def create_postional_tensor(length, batch_size, device): 
    return torch.arange(0, length, device=device).unsqueeze(0).repeat(batch_size, 1) # [0, 1, 2, 3, ..., length - 1] 

In [ ]:
# CLASE: GatedConvEncoder
# Esta es una clase que define un módulo de encoder para una arquitectura Encoder - Decoder que utiliza convoluciones. 
# Basado en convoluciones que utiliza estas embeddings posicionales y realiza convoluciones, activaciones GLU y conexiones 
# residuales para procesar la entrada y generar salidas.

# Based on https://github.com/facebookresearch/fairseq/blob/main/fairseq/

class GatedConvEncoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, kernel_size, dropout, device, max_length = max_seq_length):
        super().__init__()
        assert kernel_size % 2 == 1, "Invalid kernel size"
        
        self.scale = math.sqrt(0.5) 
        self.tok_embedding = nn.Embedding(input_dim, emb_dim)   # token embedding layer
        self.pos_embedding = nn.Embedding(max_length, emb_dim)  # positional embedding layer
        self.incoming_projection = nn.Linear(emb_dim, hid_dim)
        self.outgoing_projection = nn.Linear(hid_dim, emb_dim)
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, out_channels = 2 * hid_dim, kernel_size = kernel_size, padding = (kernel_size - 1) // 2) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout)
        self.device = device

    # x = [batch size, x len]  
    def forward(self, x):        
        batch_size, src_len = x.shape[0], x.shape[1]   
        pos = create_postional_tensor(src_len, batch_size, self.device) # [batch size, x len]       
        # Calcula las embeddings de los tokens y las embeddings posicionales.
        te = self.tok_embedding(x)      # [BS, X, emb dim]
        pe = self.pos_embedding(pos)    # [BS, X, emb dim]
        # Combina las embeddings sumándolas.
        embedded = self.dropout(te + pe)                # [BS, X, emb dim]
        # Proyecta las embeddings combinadas a una dimensión oculta.
        conv_input = self.incoming_projection(embedded) # [BS, X, hid dim]
        # Aplica capas de convolución, activación GLU y conexiones residuales.
        conv_input = conv_input.permute(0, 2, 1)        # [BS, hid dim, X]  
        for conv in self.convs:
            # Pass through convolutional layer
            conved = conv(self.dropout(conv_input))         # [BS, 2 * hid dim, X]
            # Pass through GLU activation function
            # GLU is far more stable than ReLU and learns faster than sigmoid.
            conved = F.glu(conved, dim = 1)                 # [batch size, hid dim, src len]            
            # Apply residual connection
            conved = (conved + conv_input) * self.scale     # [batch size, hid dim, src len]
            # Set conv_input to conved for next loop iteration
            conv_input = conved       
        #permute and convert back to emb dim
        conved = self.outgoing_projection(conved.permute(0, 2, 1))  # [batch size, src len, emb dim]
        combined = (conved + embedded) * self.scale                 # combined = [batch size, src len, emb dim]
        return conved, combined

In [ ]:
# Based on https://github.com/facebookresearch/fairseq/blob/main/fairseq/

class GatedConvDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, kernel_size, dropout, trg_pad_idx, device, max_length = max_seq_length):
        super().__init__()   
        self.kernel_size = kernel_size
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        self.scale = self.scale = math.sqrt(0.5)  
        self.tok_embedding = nn.Embedding(output_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
         # projects from output of convolution to embedding dimension
        self.dec_incoming_projection = nn.Linear(emb_dim, hid_dim)
        # projects from embedding dimension to convolution size
        self.dec_outgoing_projection = nn.Linear(hid_dim, emb_dim)       
        # projects from output of convolution to embedding dimension
        self.attn_incoming_projection = nn.Linear(hid_dim, emb_dim)
        # projects from embedding dimension to convolution size
        self.attn_outgoing_projection = nn.Linear(emb_dim, hid_dim) 
        # project back to size of vocabulary 
        self.fc_out = nn.Linear(emb_dim, output_dim)
        # temporal convolutions
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, out_channels = 2 * hid_dim, kernel_size = kernel_size) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout)


    # Computes attention
    # embedded          [batch size, trg len, emb dim]
    # conved            [batch size, hid dim, trg len]
    # encoder_conved    [batch size, src len, emb dim]
    # encoder_combined  [batch size, src len, emb dim]
        
    def calculate_attention(self, embedded, conved, encoder_conved, encoder_combined):
        #permute and convert back to emb dim
        conved_emb = self.attn_incoming_projection(conved.permute(0, 2, 1))             # [batch size, trg len, emb dim]
        combined = (conved_emb + embedded) * self.scale                                 # [batch size, trg len, emb dim]     
        energy = torch.matmul(combined, encoder_conved.permute(0, 2, 1))                # [batch size, trg len, src len]
        attention = F.softmax(energy, dim=2)                                            # [batch size, trg len, src len]
        attended_encoding = torch.matmul(attention, encoder_combined)                   # [batch size, trg len, emd dim]
        #convert from emb dim -> hid dim
        attended_encoding = self.attn_outgoing_projection(attended_encoding)            # [batch size, trg len, hid dim]
        #apply residual connection
        attended_combined = (conved + attended_encoding.permute(0, 2, 1)) * self.scale  # [batch size, hid dim, trg len]
        return attention, attended_combined

    # trg               [trg len, batch size]
    # encoder_conved    [batch size, src len, emb dim]
    # encoder_combined  [batch size, src len, emb dim]

    def forward(self, trg, encoder_conved, encoder_combined):              
        batch_size, trg_len = trg.shape[0], trg.shape[1]   
        #create position tensor
        pos = create_postional_tensor(trg_len, batch_size, self.device)                         # [trg len, batch size]
        #embed tokens and positions
        tok_embedded = self.tok_embedding(trg)                                                  # [trg len, batch size, emb dim]
        pos_embedded = self.pos_embedding(pos)                                                  # [trg len, batch size, emb dim]
        embedded = self.dropout(tok_embedded + pos_embedded)                                    # [batch size, trg len, emb dim]
        # project to size of convolution
        conv_input = self.dec_incoming_projection(embedded)                                     # [batch size, trg len, hid dim]      
        # permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1)                                                # [batch size, hid dim, trg len]
        batch_size = conv_input.shape[0]
        hid_dim = conv_input.shape[1]
        for conv in self.convs:
            conv_input = self.dropout(conv_input)
            padding = torch.zeros(batch_size, hid_dim, self.kernel_size - 1, device=self.device).fill_(self.trg_pad_idx)   
            padded_conv_input = torch.cat((padding, conv_input), dim = 2) # [batch size, hid dim, trg len + kernel size - 1]
            conved = conv(padded_conv_input)                              # [batch size, 2 * hid dim, trg len]
            conved = F.glu(conved, dim = 1)                               # [batch size, hid dim, trg len]
            attention, conved = self.calculate_attention(embedded, conved, encoder_conved, encoder_combined)  # [batch size, trg len, src len]
            conved = (conved + conv_input) * self.scale                   # [batch size, hid dim, trg len]
            conv_input = conved          
        conved = self.dec_outgoing_projection(conved.permute(0, 2, 1))      # [batch size, trg len, emb dim]
        output = self.fc_out(self.dropout(conved))                          # [batch size, trg len, output dim]
        return output, attention

### 1-Encode 1-Decoder Seq2Seq using CNN

In [ ]:
class GatedConvSeq2Seq(nn.Module):
    def __init__(self, name, encoder, decoder):
        super().__init__()
        self.name = name
        self.encoder = encoder
        self.decoder = decoder
    # x = [batch size, x len]
    # y = [batch size, y len - 1]
    def forward(self, x, y):
        encoder_conved, encoder_combined = self.encoder(x)
        output, attention = self.decoder(y, encoder_conved, encoder_combined) # [batch size, trg len - 1, output dim], [batch size, trg len - 1, src len]        
        return output, attention
    def getName(self):
        return self.name

### 1-Encode 2-Decoder Seq2Seq using CNN

In [ ]:
class GatedConvDualTaskSeq2Seq(nn.Module):
    def __init__(self, name, encoder, decoder_a, decoder_b):
        super().__init__()
        self.name = name
        self.encoder = encoder
        self.decoder_a = decoder_a # Decoder for Task No. 1 (EN to ES)
        self.decoder_b = decoder_b # Decoder for Task No. 2 (EN to FR)

    # x = [batch size, src len]
    # y0 = [batch size, y0 len - 1] (<bos> token sliced off without <eos>)
    # y1 = [batch size, y1 len - 1] (<bos> token sliced off without <eos>) 
    def forward(self, x, y0, y1, returns_attention = False):         
        encoder_conved, encoder_combined = self.encoder(x) # [batch size, src len, emb dim], [batch size, src len, emb dim]
        output_a, attention_a = self.decoder_a(y0, encoder_conved, encoder_combined) # [batch size, y0 len - 1, output dim], attention_a = [batch size, y0 len - 1, x len]
        output_b, attention_b = self.decoder_b(y1, encoder_conved, encoder_combined) # [batch size, y1 len - 1, output dim], attention_b = [batch size, y1 len - 1, x len]
        
        if (returns_attention):
            return output_a, output_b, attention_a, attention_b
        else:
            return output_a, output_b

    def getName(self):
        return self.name  

### CNN Model train and evaluate routines

In [ ]:

gc_count = 5

def train_single_cnn(model,  dataset, optimizer, loss_fn, src_pad_idx, tgt_pad_idx, device, clip=None, smoothing = True):
    model.train()       
    epoch_loss = 0
    epoch_accm = 0 
    epoch_wmem = 0
    epoch_lmem = 0   

    i = 0
    batches_before_gc = 0
    for  src, tgt in iter(dataset):
        
        src = src.permute(1, 0)
        tgt = tgt.permute(1, 0)
        
        tgt_input = tgt[:, :-1]

        epoch_wmem += getSizeOf(src)
        epoch_wmem += getSizeOf(tgt)

        optimizer.zero_grad()                    

        logits, _ = model(src, tgt_input)

        epoch_lmem += getSizeOf(logits)

        tgt_output = tgt[:,1:].contiguous().view(-1)
        log_output = logits.contiguous().view(-1, logits.shape[-1])

        loss = loss_fn(log_output, tgt_output)
                    
        loss.backward()

        if (not clip == None):
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step() 
            
        accm = masked_accuracy(tgt_output, log_output,  PAD_ES_IDX)

        epoch_accm += accm
        epoch_loss += loss
        
        if i % 1000 == 999: 
            print(' - train - batch {0} loss: {1} acc: {2}'.format(i + 1, epoch_loss.item() / (i + 1), epoch_accm.item() / (i + 1) ))

        epoch_wmem += getSizeOf(loss)
        epoch_wmem += getSizeOf(accm)
        i += 1
    
        del src
        del tgt
        del tgt_input
        del tgt_output
        del loss
        del accm
        del logits
        batches_before_gc += 1

        if (batches_before_gc > gc_count):
            torch.cuda.empty_cache()       
            batches_before_gc = 0

    train_loss = (epoch_loss / batches_for_training).item()
    train_accm = (epoch_accm / batches_for_training).item() 
    wrk_mem = epoch_wmem / batches_for_training
    log_mem = epoch_lmem / batches_for_training    
    return train_loss, train_accm, wrk_mem, log_mem
    
def eval_single_cnn(model,  dataset, loss_fn, src_pad_idx, tgt_pad_idx, device):
    model.eval()    
    epoch_loss = 0
    epoch_accm = 0 
    epoch_wmem = 0
    epoch_lmem = 0  
    with torch.no_grad():
        i = 0
        batches_before_gc = 0
        for  src, tgt in iter(dataset):

            src = src.permute(1, 0)
            tgt = tgt.permute(1, 0)
            
            tgt_input = tgt[:, :-1]

            epoch_wmem += getSizeOf(src)
            epoch_wmem += getSizeOf(tgt)

            logits, _ = model(src, tgt_input)

            epoch_lmem += getSizeOf(logits)

            tgt_output = tgt[:,1:].contiguous().view(-1)
            log_output = logits.contiguous().view(-1, logits.shape[-1])

            loss = loss_fn(log_output, tgt_output)

            epoch_loss += loss
            
            accm = masked_accuracy(tgt_output, log_output,  PAD_ES_IDX)
            
            epoch_accm += accm
          
            if i % 1000 == 999: 
                print(' - eval - batch {0} loss: {1} acc: {2}'.format(i + 1, epoch_loss.item() / (i + 1), epoch_accm.item() / (i + 1) ))

            epoch_wmem += getSizeOf(loss)
            epoch_wmem += getSizeOf(accm)
            i += 1

            del src
            del tgt
            del tgt_input
            del tgt_output
            del logits
            del loss
            del accm
            batches_before_gc += 1
            if (batches_before_gc > gc_count):
                torch.cuda.empty_cache()       
                batches_before_gc = 0
    
    valid_loss = (epoch_loss / batches_for_validation).item()
    valid_accm = (epoch_accm / batches_for_validation).item()
    wrk_mem = epoch_wmem / batches_for_validation
    log_mem = epoch_lmem / batches_for_validation
    return valid_loss , valid_accm, wrk_mem, log_mem



In [ ]:
import cProfile, pstats, io
from pstats import SortKey

from timeit import default_timer as timer

def run_single_cnn(model, filename, trnset, valset, optimizer, scheduler, loss_fn, src_pad_idx, tgt_pad_idx):
    
    
    print(model.getName())    
    
    now = datetime.datetime.now()
    time_str = now.strftime("%Y-%m-%d %H:%M:%S")
    print('[{0}] - Training / Testing Process - Started!'.format(time_str))

    metrics_by_epoch = {}
    
    best_valid_loss = float("inf")
    
    early_stopping_count = 0

    base_epoch = -1 
    # Check for previos training checkpoints (latest)
    for epoch in range(NUMBER_OF_EPOCHS):
        if os.path.exists(filename.format(epoch)):
            base_epoch = epoch
        else:
            break
    
    # Load the latest checkpoint 
    if (base_epoch >= 0):
        # Load model checkpoint file (latest)
        checkpoint = torch.load(filename.format(base_epoch))
        # Load model data only when match filename with epoch information
        if (checkpoint['epoch'] == base_epoch):
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            metrics_by_epoch = checkpoint['metrics_by_epoch']
            best_valid_loss = checkpoint['best_valid_loss']
            early_stopping_count = checkpoint['early_stopping_count']
            print("Continuing training model {0} from checkpoint at epoch {1}".format(model.getName(), base_epoch + 1))
            
            for i in range(1, base_epoch):
                print("- Epoch {0} achieves T.Loss = {1} E.Loss = {2}".format(i, metrics_by_epoch[i]['train_loss'], metrics_by_epoch[i]['valid_loss']))
            
            print("- best_valid_loss = {0}".format(best_valid_loss))
        else:
            raise RuntimeError("Checkpoint {0} is corrupt".format(base_epoch))

    if (base_epoch + 1 < NUMBER_OF_EPOCHS):
        for epoch in range(base_epoch + 1, NUMBER_OF_EPOCHS):

            trn_s_time = timer()       
            train_loss, train_acc, trn_wrk_mem, trn_log_mem = train_single_cnn(model, trnset, optimizer, loss_fn,  src_pad_idx, tgt_pad_idx, DEVICE, clip=CLIPPING_VALUE)
            trn_e_time = timer()
            trn_elapsed_time = trn_e_time - trn_s_time


            val_s_time = timer()  
            valid_loss, valid_acc, val_wrk_mem, val_log_mem = eval_single_cnn(model, valset, loss_fn, src_pad_idx, tgt_pad_idx, DEVICE)
            val_e_time = timer()
            val_elapsed_time = val_e_time - val_s_time

            scheduler.step(valid_loss)


            print("Model {0} at Epoch {1} Duration = {2} second(s) LR = {3}".format(model.getName(), epoch + 1, trn_elapsed_time + val_elapsed_time, scheduler.get_last_lr()))
            print("Model {0} at Epoch {1} Trn.Loss = {2} and Trn.Accuracy = {3}".format(model.getName(), epoch + 1, train_loss, train_acc))
            print("Model {0} at Epoch {1} Val.Loss = {2} and Val.Accuracy = {3}".format(model.getName(), epoch + 1, valid_loss, valid_acc))

            metrics_by_epoch[epoch + 1] = dict(
                model_name = model.getName(),
                model_epoch = epoch + 1,
                train_loss = train_loss,
                train_acc = train_acc,
                train_ppl = np.exp(train_loss),
                train_wrkmem = trn_wrk_mem,
                train_logmem = trn_log_mem,
                valid_loss = valid_loss,
                valid_acc = valid_acc,
                valid_ppl = np.exp(valid_loss),
                valid_wrkmem = val_wrk_mem,
                valid_logmem = val_log_mem,
                scheduler_lr = scheduler.get_last_lr(),
                trn_elapsed_time = trn_elapsed_time,
                val_elapsed_time = val_elapsed_time
            )


            temp_filename = "temp_" + filename.format(epoch)

            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'early_stopping_count': early_stopping_count,
            'metrics_by_epoch' : metrics_by_epoch,
            'best_valid_loss' : best_valid_loss
            }, temp_filename)

            os.rename(temp_filename, filename.format(epoch))

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                early_stopping_count = 0
            elif epoch > EARLY_STOPPING_EPOCHS:
                early_stopping_count += 1
                      
            if early_stopping_count == EARLY_STOPPING_EPOCHS:
                print("Early stopping triggered in epoch {0}".format(epoch + 1))
                break
        
            tqdm.write("Waiting for gpu cooling time...")

            let_gpu_rest(INNER_GPU_REST_TIME)

    return metrics_by_epoch

### Model testing

In [ ]:

CNN_ES_enc = GatedConvEncoder(INPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_ENC_LAYERS, GCNN_ENC_KERNEL_SIZE, GCNN_ENC_DROPOUT, DEVICE).to(DEVICE)
CNN_ES_dec = GatedConvDecoder(OUTPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, DEVICE).to(DEVICE)

CNN_FR_enc = GatedConvEncoder(INPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_ENC_LAYERS, GCNN_ENC_KERNEL_SIZE, GCNN_ENC_DROPOUT, DEVICE).to(DEVICE)
CNN_FR_dec = GatedConvDecoder(OUTPUT_FR_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, DEVICE).to(DEVICE)

CNN_model_EN_ES = GatedConvSeq2Seq("CNN_EN_ES", CNN_ES_enc, CNN_ES_dec).to(DEVICE)
CNN_model_EN_FR = GatedConvSeq2Seq("CNN_EN_FR", CNN_FR_enc, CNN_FR_dec).to(DEVICE)

CNN_optimizer_EN_ES = optim.Adam(CNN_model_EN_ES.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
CNN_optimizer_EN_FR = optim.Adam(CNN_model_EN_FR.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

scheduler_EN_ES = torch.optim.lr_scheduler.ReduceLROnPlateau(CNN_optimizer_EN_ES, 'min')
scheduler_EN_FR = torch.optim.lr_scheduler.ReduceLROnPlateau(CNN_optimizer_EN_FR, 'min')

CNN_Lossfn_EN_ES = nn.CrossEntropyLoss(ignore_index = CNN_TRG_PAD_IDX)
CNN_Lossfn_EN_FR = nn.CrossEntropyLoss(ignore_index = CNN_TRG_PAD_IDX)

CNN_DUAL_enc = GatedConvEncoder(INPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_ENC_LAYERS, GCNN_ENC_KERNEL_SIZE, GCNN_ENC_DROPOUT, DEVICE).to(DEVICE)
CNN_DUAL_ES_dec = GatedConvDecoder(OUTPUT_ES_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, DEVICE).to(DEVICE)
CNN_DUAL_FR_dec = GatedConvDecoder(OUTPUT_FR_VOCAB_DIM, GCNN_EMBDIM, GCNN_HID_DIM, GCNN_DEC_LAYERS, GCNN_DEC_KERNEL_SIZE, GCNN_DEC_DROPOUT, CNN_TRG_PAD_IDX, DEVICE).to(DEVICE)
CNN_DUAL_model = GatedConvDualTaskSeq2Seq("CNN_EN_ES_FR", CNN_DUAL_enc, CNN_DUAL_ES_dec, CNN_DUAL_FR_dec).to(DEVICE)
CNN_DUAL_optimizer = optim.Adam(CNN_DUAL_model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
CNN_DUAL_Lossfn_ES = nn.CrossEntropyLoss(ignore_index = CNN_TRG_PAD_IDX)
CNN_DUAL_Lossfn_FR = nn.CrossEntropyLoss(ignore_index = CNN_TRG_PAD_IDX)

scheduler_DUAL = torch.optim.lr_scheduler.ReduceLROnPlateau(CNN_DUAL_optimizer, 'min')

In [ ]:
random.seed(1234)
torch.manual_seed(1234)
CNN_model_EN_ES.apply(init_transformer_model)

In [ ]:
count_params(CNN_model_EN_ES)

In [ ]:
random.seed(1234)
torch.manual_seed(1234)
CNN_model_EN_FR.apply(init_transformer_model)

In [ ]:
count_params(CNN_model_EN_FR)

In [ ]:
random.seed(1234)
torch.manual_seed(1234)
CNN_DUAL_model.apply(init_transformer_model)

In [ ]:
count_params(CNN_DUAL_model)

In [ ]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity


profile = True

histories = []

random.seed(1234)
torch.manual_seed(1234)
CNN_model_EN_ES = CNN_model_EN_ES.to(DEVICE)


history = None

if (profile):
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        with record_function("model_inference"):
            history = run_single_cnn(CNN_model_EN_ES, CNN_EN_ES_model_path, Bilingual_EN_to_ES_trnset, Bilingual_EN_to_ES_valset, CNN_optimizer_EN_ES, scheduler_EN_ES, CNN_Lossfn_EN_ES, PAD_EN_IDX, PAD_ES_IDX)

        print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
        print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
else:
    history = run_single_cnn(CNN_model_EN_ES, CNN_EN_ES_model_path, Bilingual_EN_to_ES_trnset, Bilingual_EN_to_ES_valset, CNN_optimizer_EN_ES, scheduler_EN_ES, CNN_Lossfn_EN_ES, PAD_EN_IDX, PAD_ES_IDX)


name = CNN_model_EN_ES.getName()
df = pd.DataFrame.from_dict(history, orient='index')   
plot_accuracy_curve(name, df)
plot_loss_curve(name, df)
plot_pplx_curve(name, df)
histories.append(df)
torch.cuda.empty_cache()
let_gpu_rest(OUTER_GPU_REST_TIME)


random.seed(1234)
torch.manual_seed(1234)
CNN_model_EN_FR = CNN_model_EN_FR.to(DEVICE)

history = None

if (profile):
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        with record_function("model_inference"):
            history = run_single_cnn(CNN_model_EN_FR, CNN_EN_FR_model_path, Bilingual_EN_to_FR_trnset, Bilingual_EN_to_FR_valset, CNN_optimizer_EN_FR, scheduler_EN_FR, CNN_Lossfn_EN_FR, PAD_EN_IDX, PAD_FR_IDX)
        print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
        print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
else:
    history = run_single_cnn(CNN_model_EN_FR, CNN_EN_FR_model_path, Bilingual_EN_to_FR_trnset, Bilingual_EN_to_FR_valset, CNN_optimizer_EN_FR, scheduler_EN_FR, CNN_Lossfn_EN_FR, PAD_EN_IDX, PAD_FR_IDX)

name = CNN_model_EN_FR.getName()
df = pd.DataFrame.from_dict(history, orient='index')   
plot_accuracy_curve(name, df)
plot_loss_curve(name, df)
plot_pplx_curve(name, df)
histories.append(df)
torch.cuda.empty_cache()
let_gpu_rest(OUTER_GPU_REST_TIME)


result = pd.concat(histories)
result.to_csv("./metrics_gcnn_models.csv", sep=';', index=True, encoding='utf-8')

display(result)
